# CARATE-BERT

In [8]:
%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm


def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

## PCBE
Binding affinities

In [3]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit
import torch
import numpy as np 
torch.manual_seed(43)
path = '.'
dataset = MoleculeNet(path, name="pcba").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')
weights = [1/1384, 1/94]

#sampler_train = torch.utils.data.sampler.WeightedRandomSampler(weights, num_samples=len(train_dataset))

#sampler_test = torch.utils.data.sampler.WeightedRandomSampler(weights, num_samples=len(test_dataset))
y = np.zeros((len(train_dataset), dataset.num_classes))

for i in range(len(train_dataset)):
  y[i, :] = torch.nan_to_num(train_dataset[i].y)
target_list = torch.tensor(y).type(torch.LongTensor)
#target_list = target_list[torch.randperm(len(target_list))].type(torch.LongTensor)
print(y)
print(target_list==y)
class_weights = 1./torch.tensor([1384, 94], dtype=torch.float) 
class_weights_all = class_weights[target_list]
print(class_weights_all)
weighted_sampler = torch.utils.data.sampler.WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

test_loader = DataLoader(test_dataset,batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 12))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y
factor = np.zeros((1,1,12))
add = np.zeros((1,1,12))
std = np.zeros((1,1,12))
for i in range(12): 
  norm = np.linalg.norm(y[:,0,i], ord=2)
  factor[:,:,i] =  norm
  add[:,:,i] =   np.mean(y[:,0,i])
  std[:,:,i] = np.std(y[:,0,i])
print(np.mean(add))
print(factor)
print(np.mean(std))
print(y/factor)
#y_2=F.normalize(torch.from_numpy(y), p=2, dim=2)
#factor = y/y_2
#print(y/y_2)
#factor = factor[0]
print(dataset[789].y/factor[0])

## HIV

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = MoleculeNet(path, name="hiv").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

Train Dataset: HIV(37015):
Number of graphs: 37015
Number of features: 9
Number of classes: 1
Test Dataset: HIV(4112):
Number of graphs: 4112
Number of features: 9
Number of classes: 1


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6, heads = 12)
        self.conv5 = GraphConv(dim*12, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 1)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        output_pred = (output > 0.5).float()
        loss = torch.nn.BCEWithLogitsLoss(pos_weight= torch.Tensor([400]).to(device))
        loss = loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader,  epoch, test=False):
    model.eval()

    correct = 0
    auc = np.zeros(dataset.num_classes)
    if test:
      outs =[]
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        if test:
          outs.append(output_probs.cpu().detach().numpy())
        correct += (output == data.y).float().sum()
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("HIV_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch)
    test_acc = test(test_loader, epoch, True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

KeyboardInterrupt: ignored

## SIDER 

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = MoleculeNet(path, name="sider").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

Processing...
Done!
Train Dataset: SIDER(1285):
Number of graphs: 1285
Number of features: 9
Number of classes: 27
Test Dataset: SIDER(142):
Number of graphs: 142
Number of features: 9
Number of classes: 27


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 2)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        y = data.y[:, 0].type(torch.LongTensor)
        y = y.to(device)
        loss = F.nll_loss(output, y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        y = data.y[:, 0].type(torch.LongTensor)
        y = y.to(device)
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(y).sum().item()
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
        outputs =np.concatenate(outs, axis=0 ).astype(float)
        np.savetxt("Sider_epoch"+str(epoch)+".csv", outputs)

    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Epoch: 001, Train Loss: 20.9759265, Train Acc: 0.5284047, Test Acc: 0.5211268
Epoch: 002, Train Loss: 2.1344892, Train Acc: 0.5221790, Test Acc: 0.5140845
Epoch: 003, Train Loss: 0.8315162, Train Acc: 0.4350195, Test Acc: 0.4577465
Epoch: 004, Train Loss: 0.7107426, Train Acc: 0.5276265, Test Acc: 0.5281690
Epoch: 005, Train Loss: 0.7045599, Train Acc: 0.5478599, Test Acc: 0.5492958
Epoch: 006, Train Loss: 0.7118828, Train Acc: 0.4684825, Test Acc: 0.4929577
Epoch: 007, Train Loss: 0.7082243, Train Acc: 0.5610895, Test Acc: 0.5774648
Epoch: 008, Train Loss: 0.7053720, Train Acc: 0.5400778, Test Acc: 0.5704225
Epoch: 009, Train Loss: 0.6983605, Train Acc: 0.5229572, Test Acc: 0.5774648
Epoch: 010, Train Loss: 0.6980576, Train Acc: 0.5540856, Test Acc: 0.6408451
Epoch: 011, Train Loss: 0.7021216, Train Acc: 0.4801556, Test Acc: 0.4929577
Epoch: 012, Train Loss: 0.6960352, Train Acc: 0.4568093, Test Acc: 0.4788732
Epoch: 013, Train Loss: 0.6949527, Train Acc: 0.5579767, Test Acc: 0.598591

KeyboardInterrupt: ignored

## BBBP

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = MoleculeNet(path, name="bbbp").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')

count = 0


test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

Train Dataset: BBPB(1836):
Number of graphs: 1836
Number of features: 9
Number of classes: 1
Test Dataset: BBPB(203):
Number of graphs: 203
Number of features: 9
Number of classes: 1


IndexError: ignored

In [ ]:
for i in range(len(train_dataset)):
  if train_dataset[i].y[0] == 1: 
    count+=1
print("count")
print(count/len(train_dataset))
print(len(train_dataset))
print((len(train_dataset)-count)/len(train_dataset))
print(train_dataset[i].y[0].shape)

count
2.2908496732026142
1836
-1.2908496732026145
torch.Size([1])


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 2)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        y = data.y[:, 0].type(torch.LongTensor)
        y = y.to(device)
        loss = F.nll_loss(output, y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        y = data.y[:, 0].type(torch.LongTensor)
        y = y.to(device)
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(y).sum().item()
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
        outputs =np.concatenate(outs, axis=0 ).astype(float)
        np.savetxt("BPPP_epoch"+str(epoch)+".csv", outputs)

    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Epoch: 001, Train Loss: 8.4716774, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 002, Train Loss: 0.6772530, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 003, Train Loss: 0.6008534, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 004, Train Loss: 0.5906292, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 005, Train Loss: 0.5781767, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 006, Train Loss: 0.5635467, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 007, Train Loss: 0.5623611, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 008, Train Loss: 0.5539443, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 009, Train Loss: 0.5407049, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 010, Train Loss: 0.5328917, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 011, Train Loss: 0.5273293, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 012, Train Loss: 0.5264424, Train Acc: 0.7647059, Test Acc: 0.7684729
Epoch: 013, Train Loss: 0.5292502, Train Acc: 0.7647059, Test Acc: 0.7684729

KeyboardInterrupt: ignored

In [ ]:
from sklearn import metrics
y = np.zeros((len(test_dataset), dataset.num_classes))
x = np.loadtxt("BPPP_epoch499.csv")
for i in range(len(test_dataset)):
  y[i, :] = torch.nan_to_num(test_dataset[i].y)
print(y.shape)
for i in range(len(test_dataset)): 
   if y[i, 0]<y[i, 1]:
     print("class2")
   else: 
     print("class1")
print(x==y)
auch =metrics.roc_auc_score(y, x)
print(auch) 

## Clintox
have to gunzip

In [6]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit
torch.manual_seed(43)
path = '.'
dataset = MoleculeNet(path, name="clintox").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')
weights = [1/1384, 1/94]

#sampler_train = torch.utils.data.sampler.WeightedRandomSampler(weights, num_samples=len(train_dataset))

#sampler_test = torch.utils.data.sampler.WeightedRandomSampler(weights, num_samples=len(test_dataset))
y = np.zeros((len(train_dataset), dataset.num_classes))

for i in range(len(train_dataset)):
  y[i, :] = torch.nan_to_num(train_dataset[i].y)
target_list = torch.tensor(y).type(torch.LongTensor)
#target_list = target_list[torch.randperm(len(target_list))].type(torch.LongTensor)
print(y)
print(target_list==y)
class_weights = 1./torch.tensor([1384, 94], dtype=torch.float) 
class_weights_all = class_weights[target_list]
print(class_weights_all)
weighted_sampler = torch.utils.data.sampler.WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

test_loader = DataLoader(test_dataset,batch_size=15)
train_loader = DataLoader(train_dataset, batch_size=15)

Extracting ./clintox/raw/clintox.csv.gz
Processing...
Done!


Train Dataset: ClinTox(1336):
Number of graphs: 1336
Number of features: 9
Number of classes: 2
Test Dataset: ClinTox(148):
Number of graphs: 148
Number of features: 9
Number of classes: 2
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
False
tensor([[0.0007, 0.0106],
        [0.0106, 0.0007],
        [0.0106, 0.0007],
        ...,
        [0.0106, 0.0007],
        [0.0106, 0.0007],
        [0.0106, 0.0007]])


In [8]:
count = 0
for i in range(len(dataset)):
  if dataset[i].y[0,0] == 1: 
    count+=1
print(count)
print(count/len(dataset))
print(len(dataset))
print((len(dataset)-count))

1390
0.9366576819407008
1484
94


In [9]:
count = 0
for i in range(len(train_dataset)):
  if train_dataset[i].y[0,1] == 1: 
    count+=1
  if train_dataset[i].y[0,1] == 1 and train_dataset[i].y[0,0] == 1: 
    print("yeah")
print("count")
print(count/len(train_dataset))
print(len(train_dataset))
print((len(train_dataset)-count)/len(train_dataset))

yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
yeah
count
0.07559880239520958
1336
0.9244011976047904


In [10]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
from sklearn import metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        #self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv1 = GATConv(num_features, dim, dropout = 0.6, heads = 12)
        self.conv3 = GATConv(12*dim, dim, dropout = 0.6)
        #self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 2)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        #x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(loader):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']
  
    loss_all = 0
    weight = torch.tensor([100, 1]).type(torch.FloatTensor).to(device)
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        #print(output)
        #y = torch.reshape(data.y[:, 1].type(torch.FloatTensor), (len(data.y),1))
        #y = y.to(device)
        #print(data.y)
        #print((data.y[:,1]==1).sum().item())
        #print(output)
        #print(output)
        loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([1.0, 15.0]))
        loss = loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader, epoch, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    class_weights = torch.tensor([1.0, 20000.0]).to(device)
    for data in loader:
       data.x = data.x.type(torch.FloatTensor)
       data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
       data = data.to(device)
        
       optimizer.zero_grad()
       output = model(data.x, data.edge_index, data.batch)
       #y = torch.reshape(data.y[:, 1].type(torch.FloatTensor), (len(data.y),1))
       #y = y.to(device)
       #print(y)
       pred = (output>0.5).float()
       #print(pred)
       #pred = output.max(dim=1)[1]
       #pred = (output>0.4).float().to(device)
       #print(data.y)
       #print(pred)
       correct += (data.y[:,0] == pred[:,0]).sum().item()
       correct += (data.y[:,1] == pred[:,1]).sum().item()
       if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("ClinTox_epoch"+str(epoch)+".csv", outputs)
      print("in Epoch")
      print("Correct predictions epoch: ",correct)
      print("Num_predictions",  len(loader.dataset)*2)
    #print("pred was", pred)
    #print("correct was: ", y)
    return correct / (2*len(loader.dataset))
device =("cpu")
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch)
    test_acc = test(test_loader, epoch, test=True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))
    y = np.zeros((len(test_dataset), dataset.num_classes))
    x = np.loadtxt("ClinTox_epoch"+str(epoch)+".csv")
    for i in range(len(test_dataset)):
      y[i, :] = torch.nan_to_num(test_dataset[i].y)
    auc = metrics.roc_auc_score(y, x)
    print("AUC is:", auc)
#print(y.shape)
    if auc >=0.9:
      break

in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 001, Train Loss: 0.9152552, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.39963383437677313
in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 002, Train Loss: 0.9023556, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.42466436821403497
in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 003, Train Loss: 0.9016466, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.43126039833434315
in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 004, Train Loss: 0.9017421, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.43222930671359744
in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 005, Train Loss: 0.9013018, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.43372233922852776
in Epoch
Correct predictions epoch:  275
Num_predictions 296
Epoch: 006, Train Loss: 0.9028456, Train Acc: 0.9307635, Test Acc: 0.9290541
AUC is: 0.43983391515920867
in E

In [ ]:
from sklearn import metrics
y = np.zeros((len(test_dataset), dataset.num_classes))
x = np.loadtxt("ClinTox_epoch240.csv")
print(x)
for i in range(len(test_dataset)):
  y[i, :] = torch.nan_to_num(test_dataset[i].y)
#print(y.shape)
#for i in range(len(test_dataset)): 
   #if y[i, 0]<y[i, 1]:
     #print("class2")
   #else: 
     #print("class1")
print(x==y)
auch =metrics.roc_auc_score(y, x)
print(auch) 

In [ ]:
counter_equal = 0
counter_neq = 0
for i in range(len(dataset)): 
  if dataset[i].y[0, 0] == dataset[i].y[0, 1]:
    counter_equal += 0
  else: 
    counter_neq +=1
print("conditional counts", counter_equal)
print("non conditional", counter_neq)

not conditional counts 0
conditional 1460


##Tox21 Molecule Net
have to gununzip on first download

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = MoleculeNet(path, name="Tox21").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from sklearn import metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/12
    return correct / len(train_loader.dataset)


def test(loader,  epoch, test=False):
    model.eval()

    correct = 0
    auc = np.zeros(dataset.num_classes)
    if test:
      outs =[]
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        if test:
          outs.append(output_probs.cpu().detach().numpy())
        correct += (output == data.y).float().sum()/12
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("Tox21_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch)
    test_acc = test(test_loader, test=True, epoch=epoch)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

In [ ]:
y = np.zeros((len(test_dataset), dataset.num_classes))
x = np.loadtxt("Tox21_epoch499.csv")
for i in range(len(test_dataset)):
  y[i, :] = torch.nan_to_num(test_dataset[i].y)
print(y.shape)
print(x.shape)
auch =metrics.roc_auc_score(y, x)
print(auch) 

(783, 12)
(783, 12)
0.8196557682164346


In [ ]:
import sklearn.metrics
metrics.roc_auc_score()

## Single Class 

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.6).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()
    return correct / len(train_loader.dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        print(data.y[:][0])
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (output == data.y).float().sum()/12
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0.,

KeyboardInterrupt: ignored

# Yeast

In [11]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="Yeast").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Train Dataset: Yeast(71641):
Number of graphs: 71641
Number of features: 74
Number of classes: 2
Test Dataset: Yeast(7960):
Number of graphs: 7960
Number of features: 74
Number of classes: 2


/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [12]:
count = 0
for i in range(len(dataset)):
  if dataset[i].y == 1: 
    count+=1
print(count)
print(count/len(dataset))
print(len(dataset))
print((len(dataset)-count))

9568
0.12019949498121883
79601
70033


In [13]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

def train(epoch):
    model.train()

    if epoch%51 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Epoch: 001, Train Loss: 0.3931239, Train Acc: 0.8805153, Test Acc: 0.8733668
Epoch: 002, Train Loss: 0.3510976, Train Acc: 0.8813389, Test Acc: 0.8747487
Epoch: 003, Train Loss: 0.3411380, Train Acc: 0.8828604, Test Acc: 0.8758794
Epoch: 004, Train Loss: 0.3389193, Train Acc: 0.8831116, Test Acc: 0.8760050
Epoch: 005, Train Loss: 0.3365842, Train Acc: 0.8831395, Test Acc: 0.8763819
Epoch: 006, Train Loss: 0.3358207, Train Acc: 0.8828185, Test Acc: 0.8760050
Epoch: 007, Train Loss: 0.3356574, Train Acc: 0.8822602, Test Acc: 0.8755025
Epoch: 008, Train Loss: 0.3346737, Train Acc: 0.8832931, Test Acc: 0.8766332
Epoch: 009, Train Loss: 0.3342293, Train Acc: 0.8824416, Test Acc: 0.8757538
Epoch: 010, Train Loss: 0.3328518, Train Acc: 0.8829302, Test Acc: 0.8762563
Epoch: 011, Train Loss: 0.3330555, Train Acc: 0.8832791, Test Acc: 0.8765075
Epoch: 012, Train Loss: 0.3318544, Train Acc: 0.8836002, Test Acc: 0.8770101
Epoch: 013, Train Loss: 0.3332439, Train Acc: 0.8836141, Test Acc: 0.8771357

#YeatsH

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="YeastH").shuffle()

train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 1001):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Extracting ./YeastH/YeastH.zip
Processing...
Done!
Train Dataset: YeastH(71641):
Number of graphs: 71641
Number of features: 75
Number of classes: 2
Test Dataset: Tox21(783):
Number of graphs: 783
Number of features: 9
Number of classes: 12


ValueError: ignored

In [ ]:
# Enzymes

# Enzymes

In [14]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="ENZYMES").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Train Dataset: ENZYMES(540):
Number of graphs: 540
Number of features: 3
Number of classes: 6
Test Dataset: ENZYMES(60):
Number of graphs: 60
Number of features: 3
Number of classes: 6


In [15]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6, heads=16)
        self.conv5 = GraphConv(dim*16, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        #assert len(data.y) == len(train_loader.dataset), (str(len(train_loader.dataset))+" "+str(len(data.y)))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/6
    return correct / len(train_loader.dataset)


def test(loader, epoch, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (torch.argmax(output, dim=1) == data.y).float().sum()
        #print(output, data.y)
        if test: 
          outs.append(output.cpu().detach().numpy())
    #if test: 
      #outputs =np.concatenate(outs, axis=0 ).astype(float)
      #np.savetxt("Enzymes_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch=epoch)
    test_acc = test(test_loader, epoch, test=True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))
    #y = np.zeros((len(test_dataset)))
    #x = np.loadtxt("Enzymes_epoch"+str(epoch)+".csv")
    #for i in range(len(test_dataset)):
      #y[i] = test_dataset[i].y
    y = torch.as_tensor(y)
    y = F.one_hot(y.long(), num_classes = 6).long()
    store_auc = 0
    for i in range(len(x[0,:])): 
      auc = metrics.roc_auc_score(y[:,i], x[:,i])
      print("AUC of "+str(i) +"is:", auc)
      store_auc += auc
    print("Average auc", store_auc/6)
#print(y.shape)
    if auc >=0.9:
      break

Epoch: 001, Train Loss: 0.7438272, Train Acc: 0.1648148, Test Acc: 0.1833333


ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
import random
n_classes = 5
n_samples = 10

# Create list n_samples random labels (can also be numpy array)
labels = [random.randrange(n_classes) for _ in range(n_samples)]
# Convert to torch Tensor
labels_tensor = torch.as_tensor(labels)
print(labels_tensor)
# Create one-hot encodings of labels
one_hot = torch.nn.functional.one_hot(labels_tensor, num_classes=n_classes)
print(one_hot)

tensor([3, 3, 2, 4, 1, 3, 0, 1, 4, 4])
tensor([[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1]])


# Proteins


In [12]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="PROTEINS").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Extracting ./PROTEINS/PROTEINS.zip
Processing...


Train Dataset: PROTEINS(1002):
Number of graphs: 1002
Number of features: 0
Number of classes: 2
Test Dataset: PROTEINS(111):
Number of graphs: 111
Number of features: 0
Number of classes: 2


Done!
/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = F.one_hot(data.y, num_classes = 2).type(torch.FloatTensor)
        #assert len(data.y) == len(train_loader.dataset), (str(len(train_loader.dataset))+" "+str(len(data.y)))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/6
    return correct / len(train_loader.dataset)


def test(loader, epoch, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (torch.argmax(output, dim=1) == data.y).float().sum()
        #print(output, data.y)
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("Enzymes_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch=epoch)
    test_acc = test(test_loader, epoch, test=True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))
    y = np.zeros((len(test_dataset)))
    x = np.loadtxt("Enzymes_epoch"+str(epoch)+".csv")
    for i in range(len(test_dataset)):
      y[i] = test_dataset[i].y
    y = torch.as_tensor(y)
    y = F.one_hot(y.long(), num_classes = 6).long()
    store_auc = 0
    for i in range(len(x[0,:])): 
      auc = metrics.roc_auc_score(y[:,i], x[:,i])
      print("AUC of "+str(i) +"is:", auc)
      store_auc += auc
    print("Average auc", store_auc/2)
#print(y.shape)
    if auc >=0.8:
      break

Epoch: 001, Train Loss: 0.1889554, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 002, Train Loss: 0.1886228, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 003, Train Loss: 0.1944444, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 004, Train Loss: 0.1957751, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 005, Train Loss: 0.2049235, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 006, Train Loss: 0.2070858, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 007, Train Loss: 0.2095808, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 008, Train Loss: 0.2080838, Train Acc: 0.6007984, Test Acc: 0.5495495
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5


In [ ]:
for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

#MCF-7

Breast Cancer actives to MCF-7 cells



In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="MCF-7").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Extracting ./MCF-7/MCF-7.zip
Processing...
Done!
Train Dataset: MCF-7(24993):
Number of graphs: 24993
Number of features: 46
Number of classes: 2
Test Dataset: MCF-7(2777):
Number of graphs: 2777
Number of features: 46
Number of classes: 2


In [ ]:
count = 0
for i in range(len(train_dataset)):
  if train_dataset[i].y[0] == 1: 
    count+=1
print("count")
print(count/len(train_dataset))
print(len(train_dataset))
print((len(train_dataset)-count)/len(train_dataset))
print(train_dataset[i].y[0].shape)

count
0.08318329132157004
24993
0.91681670867843
torch.Size([])


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = F.one_hot(data.y, num_classes = 2).type(torch.FloatTensor)
        #assert len(data.y) == len(train_loader.dataset), (str(len(train_loader.dataset))+" "+str(len(data.y)))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/6
    return correct / len(train_loader.dataset)


def test(loader, epoch, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (torch.argmax(output, dim=1) == data.y).float().sum()
        #print(output, data.y)
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("MCF-7_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch=epoch)
    test_acc = test(test_loader, epoch, test=True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))
    y = np.zeros((len(test_dataset)))
    x = np.loadtxt("MCF-7_epoch"+str(epoch)+".csv")
    for i in range(len(test_dataset)):
      y[i] = test_dataset[i].y
    y = torch.as_tensor(y)
    y = F.one_hot(y.long(), num_classes = 2).long()
    store_auc = 0
    for i in range(len(x[0,:])): 
      auc = metrics.roc_auc_score(y[:,i], x[:,i])
      print("AUC of "+str(i) +"is:", auc)
      store_auc += auc
    print("Average auc", store_auc/2)
#print(y.shape)
    if auc >=0.9:
      break

Epoch: 001, Train Loss: 0.3054458, Train Acc: 0.9168167, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 002, Train Loss: 0.3055991, Train Acc: 0.9168167, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 003, Train Loss: 0.3055591, Train Acc: 0.9168167, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 004, Train Loss: 0.3057058, Train Acc: 0.9170168, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 005, Train Loss: 0.3056390, Train Acc: 0.9171768, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 006, Train Loss: 0.3056858, Train Acc: 0.9167767, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 007, Train Loss: 0.3056658, Train Acc: 0.9171368, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5
Epoch: 008, Train Loss: 0.3059859, Train Acc: 0.9169368, Test Acc: 0.9225783
AUC of 0is: 0.5
AUC of 1is: 0.5
Average auc 0.5


KeyboardInterrupt: ignored

#MOLT-4
Leucemia

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="MCF-7").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Train Dataset: MCF-7(24993):
Number of graphs: 24993
Number of features: 46
Number of classes: 2
Test Dataset: MCF-7(2777):
Number of graphs: 2777
Number of features: 46
Number of classes: 2


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = F.one_hot(data.y, num_classes = 2).type(torch.FloatTensor)
        #assert len(data.y) == len(train_loader.dataset), (str(len(train_loader.dataset))+" "+str(len(data.y)))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/6
    return correct / len(train_loader.dataset)


def test(loader, epoch, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (torch.argmax(output, dim=1) == data.y).float().sum()
        #print(output, data.y)
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("MOLT_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader, epoch=epoch)
    test_acc = test(test_loader, epoch, test=True)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))
    y = np.zeros((len(test_dataset)))
    x = np.loadtxt("MCF-7_epoch"+str(epoch)+".csv")
    for i in range(len(test_dataset)):
      y[i] = test_dataset[i].y
    y = torch.as_tensor(y)
    y = F.one_hot(y.long(), num_classes = 2).long()
    store_auc = 0
    for i in range(len(x[0,:])): 
      auc = metrics.roc_auc_score(y[:,i], x[:,i])
      print("MOLT of "+str(i) +"is:", auc)
      store_auc += auc
    print("Average auc", store_auc/2)
#print(y.shape)
    if auc >=0.9:
      break

Epoch: 001, Train Loss: 0.3056255, Train Acc: 0.9179770, Test Acc: 0.9121354
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 002, Train Loss: 0.3059923, Train Acc: 0.9179770, Test Acc: 0.9121354
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 003, Train Loss: 0.3059790, Train Acc: 0.9184172, Test Acc: 0.9135758
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 004, Train Loss: 0.3061123, Train Acc: 0.9184972, Test Acc: 0.9139359
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 005, Train Loss: 0.3061191, Train Acc: 0.9182571, Test Acc: 0.9128556
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 006, Train Loss: 0.3059189, Train Acc: 0.9186572, Test Acc: 0.9124955
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 007, Train Loss: 0.3062590, Train Acc: 0.9184972, Test Acc: 0.9124955
MOLT of 0is: 0.5
MOLT of 1is: 0.5
Average auc 0.5
Epoch: 008, Train Loss: 0.3063257, Train Acc: 0.9184972, Test Acc: 0.9124955
MOLT of 0is: 0.5
MOLT of 1is: 0.5


KeyboardInterrupt: ignored

# AIDS


# Mutagenicity

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="Mutagenicity").shuffle()

train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

Extracting ./Mutagenicity/Mutagenicity.zip
Processing...
Done!
Train Dataset: Mutagenicity(3904):
Number of graphs: 3904
Number of features: 14
Number of classes: 2
Test Dataset: P388H(4147):
Number of graphs: 4147
Number of features: 73
Number of classes: 2


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 1001):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

RuntimeError: ignored

In [14]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="AIDS").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

Extracting ./AIDS/AIDS.zip
Processing...


Train Dataset: AIDS(1800):
Number of graphs: 1800
Number of features: 38
Number of classes: 2
Test Dataset: AIDS(200):
Number of graphs: 200
Number of features: 38
Number of classes: 2


Done!
/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [16]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Epoch: 001, Train Loss: -0.7594726, Train Acc: 0.8150000, Test Acc: 0.8000000
Epoch: 002, Train Loss: -0.9819686, Train Acc: 0.9288889, Test Acc: 0.9350000
Epoch: 003, Train Loss: -0.9960505, Train Acc: 0.9888889, Test Acc: 0.9800000
Epoch: 004, Train Loss: -0.9981298, Train Acc: 0.9966667, Test Acc: 0.9850000
Epoch: 005, Train Loss: -0.9982875, Train Acc: 0.9855556, Test Acc: 0.9850000
Epoch: 006, Train Loss: -0.9993216, Train Acc: 0.9477778, Test Acc: 0.9500000
Epoch: 007, Train Loss: -0.9987758, Train Acc: 0.8872222, Test Acc: 0.8850000
Epoch: 008, Train Loss: -0.9985167, Train Acc: 0.7044444, Test Acc: 0.7650000
Epoch: 009, Train Loss: -0.9992791, Train Acc: 0.5844444, Test Acc: 0.6200000
Epoch: 010, Train Loss: -0.9997485, Train Acc: 0.4527778, Test Acc: 0.5000000
Epoch: 011, Train Loss: -0.9989326, Train Acc: 0.3727778, Test Acc: 0.3900000
Epoch: 012, Train Loss: -0.9994909, Train Acc: 0.3205556, Test Acc: 0.3250000
Epoch: 013, Train Loss: -0.9994739, Train Acc: 0.2905556, Test A

KeyboardInterrupt: 

# Alchemy Full

In [17]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="alchemy_full").shuffle()
test_dataset = dataset[:len(dataset) // 10][:200]
train_dataset = dataset[len(dataset) // 10:][:2000]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

Extracting ./alchemy_full/alchemy_full.zip
Processing...


Train Dataset: alchemy_full(2000):
Number of graphs: 2000
Number of features: 6
Number of classes: 12
Test Dataset: alchemy_full(200):
Number of graphs: 200
Number of features: 6
Number of classes: 12


Done!


In [19]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, torch.max(data.y, 1)[1])
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += (output == data.y).float().sum()
    return (correct / len(loader.dataset))*100000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Epoch: 001, Train Loss: 0.6483112, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 002, Train Loss: 0.0043603, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 003, Train Loss: 0.0004202, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 004, Train Loss: 0.0002546, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 005, Train Loss: 0.0002504, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 006, Train Loss: 0.0002101, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 007, Train Loss: 0.0003078, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 008, Train Loss: 0.0001261, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 009, Train Loss: 0.0000733, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 010, Train Loss: 0.0002628, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 011, Train Loss: 0.0000723, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 012, Train Loss: 0.0000730, Train Acc: 0.0000000, Test Acc: 0.0000000
Epoch: 013, Train Loss: 0.0000780, Train Acc: 0.0000000, Test Acc: 0.0000000

#P388H

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="P388H").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        print(data.y.shape)
        print(data.y)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                       train_acc, test_acc))

Extracting ./P388H/P388H.zip
Processing...
Done!
Train Dataset: P388H(37325):
Number of graphs: 37325
Number of features: 73
Number of classes: 2
Test Dataset: P388H(4147):
Number of graphs: 4147
Number of features: 73
Number of classes: 2
torch.Size([128])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
torch.Size([128])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0,

KeyboardInterrupt: ignored

# Regression


## ZINC

In [20]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="ZINC_full").shuffle()

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Extracting ./ZINC_full/ZINC_full.zip
Processing...
Done!


Train Dataset: ZINC_full(224511):
Number of graphs: 224511
Number of features: 28
Number of classes: 247313
Test Dataset: ZINC_full(24945):
Number of graphs: 24945
Number of features: 28
Number of classes: 247313


In [21]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 5))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y
factor = np.zeros((5))
for i in range(5): 
  norm = np.linalg.norm(y[:,0,i], ord=2)
  factor[i] = norm
print(factor)
#y_2=F.normalize(torch.from_numpy(y), p=2, dim=2)
#factor = y/y_2
print(y/factor)
print(dataset[789].y/factor)

[1023.10146437 1023.10146437 1023.10146437 1023.10146437 1023.10146437]
[[[ 0.00339956  0.00339956  0.00339956  0.00339956  0.00339956]]

 [[ 0.00045221  0.00045221  0.00045221  0.00045221  0.00045221]]

 [[ 0.00224007  0.00224007  0.00224007  0.00224007  0.00224007]]

 ...

 [[-0.00296831 -0.00296831 -0.00296831 -0.00296831 -0.00296831]]

 [[ 0.00220312  0.00220312  0.00220312  0.00220312  0.00220312]]

 [[-0.00018526 -0.00018526 -0.00018526 -0.00018526 -0.00018526]]]
tensor([0.0019, 0.0019, 0.0019, 0.0019, 0.0019], dtype=torch.float64)


In [22]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 1)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch).flatten()
        loss = torch.nn.MSELoss()
        #print(data.y.shape, output_probs.shape)
        loss = loss(output_probs, data.y)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mae/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return mae/len(loader)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
          'Train Mae at epoch: {:.7f}, Test MAE at epoch: {:.7f}'.format(epoch, train_loss,
                                                  train_acc, test_acc))
    torch.cuda.empty_cache()

/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([63])) that is different to the input size (torch.Size([63, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([49])) that is different to the input size (torch.Size([49, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they hav

Epoch: 001, Train Loss at epoch: 0.0523849, Train Mae at epoch: 0.0159952, Test MAE at epoch: 0.0159488
Epoch: 002, Train Loss at epoch: 0.0094686, Train Mae at epoch: 0.0088941, Test MAE at epoch: 0.0088620
Epoch: 003, Train Loss at epoch: 0.0059284, Train Mae at epoch: 0.0085758, Test MAE at epoch: 0.0085602
Epoch: 004, Train Loss at epoch: 0.0032214, Train Mae at epoch: 0.0049874, Test MAE at epoch: 0.0050021
Epoch: 005, Train Loss at epoch: 0.0016329, Train Mae at epoch: 0.0017956, Test MAE at epoch: 0.0018015
Epoch: 006, Train Loss at epoch: 0.0013646, Train Mae at epoch: 0.0015450, Test MAE at epoch: 0.0015472
Epoch: 007, Train Loss at epoch: 0.0012989, Train Mae at epoch: 0.0015047, Test MAE at epoch: 0.0015063
Epoch: 008, Train Loss at epoch: 0.0012520, Train Mae at epoch: 0.0015416, Test MAE at epoch: 0.0015424
Epoch: 009, Train Loss at epoch: 0.0012196, Train Mae at epoch: 0.0015480, Test MAE at epoch: 0.0015489
Epoch: 010, Train Loss at epoch: 0.0011943, Train Mae at epoch: 

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 1)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch).flatten()
        loss = torch.nn.MSELoss()
        #print(data.y.shape, output_probs.shape)
        loss = loss(output_probs, data.y)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mae/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return mae/len(loader)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
          'Train Mae at epoch: {:.7f}, Test MAE at epoch: {:.7f}'.format(epoch, train_loss,
                                                  train_acc, test_acc))
    torch.cuda.empty_cache()

Epoch: 001, Train Loss at epoch: 0.0345619, Train Mae at epoch: 0.0047388, Test MAE at epoch: 0.0047347
Epoch: 002, Train Loss at epoch: 0.0054881, Train Mae at epoch: 0.0051018, Test MAE at epoch: 0.0051030
Epoch: 003, Train Loss at epoch: 0.0028600, Train Mae at epoch: 0.0025184, Test MAE at epoch: 0.0025271
Epoch: 004, Train Loss at epoch: 0.0016471, Train Mae at epoch: 0.0018567, Test MAE at epoch: 0.0018624
Epoch: 005, Train Loss at epoch: 0.0013558, Train Mae at epoch: 0.0016192, Test MAE at epoch: 0.0016213
Epoch: 006, Train Loss at epoch: 0.0012696, Train Mae at epoch: 0.0016245, Test MAE at epoch: 0.0016257
Epoch: 007, Train Loss at epoch: 0.0012161, Train Mae at epoch: 0.0016477, Test MAE at epoch: 0.0016489
Epoch: 008, Train Loss at epoch: 0.0011840, Train Mae at epoch: 0.0016674, Test MAE at epoch: 0.0016690
Epoch: 009, Train Loss at epoch: 0.0011621, Train Mae at epoch: 0.0017018, Test MAE at epoch: 0.0017038
Epoch: 010, Train Loss at epoch: 0.0011392, Train Mae at epoch: 

## Alchemy

In [1]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import torch
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="alchemy_full").shuffle()
train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

/home/sail-dev/carate/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Train Dataset: alchemy_full(182322):
Number of graphs: 182322
Number of features: 6
Number of classes: 12
Test Dataset: alchemy_full(20257):
Number of graphs: 20257
Number of features: 6
Number of classes: 12


/home/sail-dev/carate/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 12))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y
factor = np.zeros((1,1,12))
add = np.zeros((1,1,12))
std = np.zeros((1,1,12))
for i in range(12): 
  norm = np.linalg.norm(y[:,0,i], ord=2)
  factor[:,:,i] =  norm
  add[:,:,i] =   np.mean(y[:,0,i])
  std[:,:,i] = np.std(y[:,0,i])
print(np.mean(add))
print(factor)
print(np.mean(std))
print(y/factor)
#y_2=F.normalize(torch.from_numpy(y), p=2, dim=2)
#factor = y/y_2
#print(y/y_2)
#factor = factor[0]
print(dataset[789].y/factor[0])

-15.716988681808667
[[[8.61489461e+01 2.83672304e-02 1.08626915e+02 2.28857479e+05
   1.03899945e+02 2.28836977e+05 4.21827207e+04 2.28841587e+05
   2.28836566e+05 2.03088601e+01 1.39834646e+03 7.75887194e+05]]]
72.92028404696889
[[[ 0.00232084  0.00223145  0.00230429 ...  0.00280002  0.00035724
    0.00247841]]

 [[ 0.00168596  0.00206224  0.00131936 ... -0.00344752  0.00298618
    0.00187302]]

 [[ 0.0018958   0.00220325  0.00238377 ...  0.00063235  0.00308434
    0.00233248]]

 ...

 [[ 0.00272549  0.00233368  0.00292773 ...  0.00329034  0.00127579
    0.00206272]]

 [[ 0.00216954  0.00219972  0.00214759 ...  0.00147888  0.00115441
    0.00196547]]

 [[ 0.00218823  0.00215389  0.00254414 ...  0.00148791  0.00330063
    0.00195483]]]
tensor([[ 0.0028,  0.0028,  0.0027, -0.0027, -0.0023, -0.0027,  0.0023, -0.0027,
         -0.0027,  0.0026,  0.0008,  0.0025]], dtype=torch.float64)


In [5]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch % 51 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = (data.y)/factor
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        loss = torch.nn.MSELoss()
       # print(data.y.shape)
        loss = loss(output_probs, data.y[0,:,:])
        #loss_mae = torch.nn.L1Loss()
        #mae += loss_mae(output_probs, data.y).item()
        mse = loss.item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mse/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    mse = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        mse += loss.item()
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return (mae/len(loader), mse/len(loader))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 500):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
          'Train Mae at epoch: {}, Test MAE at epoch: {}'.format(epoch, train_loss,
                                                  train_acc, test_acc))
    torch.cuda.empty_cache()

Epoch: 001, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.0004741022396791352, 6.75903038716655e-07), Test MAE at epoch: (0.00047529300040204066, 6.782475915916514e-07)
Epoch: 002, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.00047407736119570024, 6.765610797115406e-07), Test MAE at epoch: (0.0004752252129903974, 6.789618319290476e-07)
Epoch: 003, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.0004713877834566493, 6.692375298700957e-07), Test MAE at epoch: (0.0004726633899044598, 6.718034796724932e-07)
Epoch: 004, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.0004854811862342043, 6.842523748090957e-07), Test MAE at epoch: (0.00048676539645442144, 6.867184580310683e-07)
Epoch: 005, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.00048294087088628936, 6.949130579973144e-07), Test MAE at epoch: (0.00048439709896777025, 6.978189945406863e-07)
Epoch: 006, Train Loss at epoch: 0.0000000, Train Mae at epoch: (0.00048053158319984434, 6.940153828659409

In [ ]:
val = 0.0004674
for i in range(len(train_dataset)):
  y[i, :, :] = train_dataset[i].y/factor

for i in range(12):
  print(abs(val/np.amax(y[:,0,i])))

In [ ]:
val_m = 0
for i in range(12):
  print(abs(val/np.amax(y[:,0,i])))
  val += abs(val/np.amax(y[:,0,i]))
val_12

In [ ]:
y

## QM-9

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="QM9").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]


In [ ]:
print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)

Train Dataset: QM9(116490):
Number of graphs: 116490
Number of features: 0
Number of classes: 19
Test Dataset: QM9(12943):
Number of graphs: 12943
Number of features: 0
Number of classes: 19


In [ ]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 19))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y
factor = np.zeros((19))
for i in range(19): 
  norm = np.linalg.norm(y[:,0,i], ord=2)
  factor[i] = norm
print(factor)
#y_2=F.normalize(torch.from_numpy(y), p=2, dim=2)
#factor = y/y_2
print(y/factor)
#factor = factor[0]
print(dataset[789].y/factor)

[1.10702396e+03 2.72470975e+04 2.36137043e+03 4.71898827e+02
 2.50781796e+03 4.40173228e+05 1.49499767e+03 4.04197394e+06
 4.04189109e+06 4.04188189e+06 4.04230007e+06 1.14752584e+04
 2.76306257e+04 2.78005238e+04 2.79601507e+04 2.57089955e+04
 6.62096125e+05 7.68474920e+02 5.69364761e+02]
[[[ 1.80339367e-03  2.38263904e-03 -3.21622411e-03 ...  9.15632881e-06
    2.07205196e-03  2.21490708e-03]]

 [[ 2.30455709e-03  2.73607136e-03 -2.46604086e-03 ...  3.36469280e-06
    2.13520299e-03  1.95995614e-03]]

 [[ 4.13071471e-03  2.87700367e-03 -2.94081116e-03 ...  4.79596525e-06
    1.86983337e-03  2.24102378e-03]]

 ...

 [[ 1.31577997e-03  3.34751249e-03 -2.63658932e-03 ...  5.50702500e-06
    1.00306458e-03  1.28044459e-03]]

 [[ 5.57196625e-03  2.78158072e-03 -2.83018513e-03 ...  5.45642819e-06
    1.53852776e-03  2.05950583e-03]]

 [[ 2.12000830e-03  2.94490075e-03 -2.71264493e-03 ...  2.93916536e-06
    2.49099872e-03  2.08748427e-03]]]
tensor([[ 2.4751e-03,  2.2924e-03, -2.8359e-03, -

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GATConv(dim, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch % 51 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        loss_mae = torch.nn.L1Loss()
        #mae += loss_mae(output_probs, data.y).item()
        mse = loss.item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mse/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return mae/len(loader)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
          'Train Mae at epoch: {:.7f}, Test MAE at epoch: {:.7f}'.format(epoch, train_loss,
                                                  train_acc, test_acc))
    torch.cuda.empty_cache()

RuntimeError: ignored

In [ ]:
val = 0.0004674
for i in range(len(train_dataset)):
  y[i, :, :] = train_dataset[i].y

for i in range(12):
  print(abs(val/np.amax(y[:,0,i])))

0.0013021301745470685
4.4665723312540075
0.001149414683179826
1.345966467375715e-06
0.006236937837500347
1.3461162329045604e-06
2.8699843268999417e-06
1.3460873655495797e-06
1.3461199005669944e-06
0.004641981063331782
3.119193433880219e-05
8.95305235762369e-08


### Whole Dataset

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="QM9").shuffle()
test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]


In [ ]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 12))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y

y_2=F.normalize(torch.from_numpy(y), p=2, dim=0)
factor = y/y_2
print(y/y_2)
factor = factor[0]
print(dataset[789].y/factor)

test_dataset = dataset[:len(dataset) // 10]
train_dataset = dataset[len(dataset) // 10:]

print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=16)
train_loader = DataLoader(train_dataset, batch_size=16)

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mae/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return mae/len(loader)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)

for epoch in range(1, 5000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
          'Train Mae at epoch: {:.7f}, Test MAE at epoch: {:.7f}'.format(epoch, train_loss,
                                                  train_acc, test_acc))
    torch.cuda.empty_cache()

ZeroDivisionError: ignored